# LangChain + uAgents in Jupyter Notebook

This notebook demonstrates running a uAgent directly in Jupyter. Note that local port access may be limited depending on your environment.

## Installation

First, let's install the necessary packages.

In [ ]:
pip install langchain-openai langchain langchain-uagents nest-asyncio ipywidgets

## Setting up Environment Variables

To use this integration, you'll need API keys for Agentverse and OpenAI.

In [10]:
import os
import time
import asyncio
import threading
import nest_asyncio
import getpass
import logging
from dotenv import load_dotenv
from IPython.display import clear_output, display, HTML
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_uagents import UAgentRegisterTool, cleanup_uagent

# Set up logging
logging.basicConfig(level=logging.INFO)

# Apply nest_asyncio to allow nested event loops - CRITICAL FOR JUPYTER
nest_asyncio.apply()

# Load environment variables
load_dotenv()

# Set up API keys
if not os.environ.get("AV_API_KEY"):
    os.environ["AV_API_KEY"] = getpass.getpass("Agentverse API Key: ")
    
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")
    
API_TOKEN = os.environ.get("AV_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

print("API keys loaded successfully!")

API keys loaded successfully!


## Agent Runner Class

This helper class manages the agent lifecycle in Jupyter. Note that local port access may be limited in many environments.
If you are running it using a .py script you don't have to do this step.

In [13]:
# Create an AgentRunner class with fixed event loop handling
class AgentRunner:
    def __init__(self):
        self.running = False
        self.agent_name = None
        self.stop_event = threading.Event()
        self.agent_info = None
        self.thread = None
    
    async def _run_agent(self, agent_obj, name, port, description, api_token):
        """Async method to run the agent using the UAgentRegisterTool"""
        self.agent_name = name
        self.running = True
        print(f"Starting agent '{name}' on port {port}...")
        
        # Create and register the agent
        tool = UAgentRegisterTool()
        try:
            self.agent_info = tool.invoke({
                "agent_obj": agent_obj,
                "name": name,
                "port": port,
                "description": description,
                "api_token": api_token
            })
            
            agent_address = self.agent_info.get('address', 'unknown')
            agent_port = self.agent_info.get('port', port)
            
            # Display info in formatted box
            clear_output(wait=True)
            display(HTML(f"""
            <div style="padding: 10px; border: 2px solid #4CAF50; border-radius: 5px; margin: 10px 0;">
                <h3 style="color: #4CAF50;">✅ Agent Successfully Created!</h3>
                <p><b>Name:</b> {self.agent_info['name']}</p>
                <p><b>Address:</b> {agent_address}</p>
                <p><b>Port:</b> {agent_port}</p>
                <p><b>Inspector URL:</b> <a href="https://agentverse.ai/agent/{agent_address}" target="_blank">https://agentverse.ai/agent/{agent_address}</a></p>
                <p><b>Local Inspector:</b> <a href="https://agentverse.ai/inspect/?uri=http://127.0.0.1:{agent_port}&address={agent_address}" target="_blank">Open Local Inspector</a></p>
                <p style="color:#f44336;"><b>Note:</b> The Local Inspector likely won't work in Jupyter as local ports are not publicly accessible.<br>Use the Agentverse URL instead to interact with your agent.</p>
                <p><b>Console Logs:</b></p>
                <pre style="background-color: #f5f5f5; padding: 8px; border-radius: 4px; max-height: 200px; overflow-y: auto;">
Agent '{name}' running on port {agent_port}
Address: {agent_address}
Registration successful!                
                </pre>
                <p style="color:#666; font-style:italic;">The agent is running in the background. Use stop_agent() to terminate it.</p>
            </div>
            """))
            
            # Keep running until stopped
            counter = 0
            while self.running and not self.stop_event.is_set():
                status = "⏳ " + "●" * (counter % 4) + "○" * (3 - (counter % 4))
                print(f"\r{status} Agent running... (Run stop_agent() in a new cell to terminate)", end="")
                counter += 1
                await asyncio.sleep(1)
        except Exception as e:
            print(f"Error creating agent: {str(e)}")
            self.running = False
        finally:
            # Clean up if stopped while running
            if self.running:
                self.running = False
                try:
                    cleanup_uagent(self.agent_name)
                except Exception as e:
                    print(f"Error during cleanup: {str(e)}")
    
    def start_agent(self, agent_obj, name, port, description, api_token):
        """Start the agent in the current event loop"""
        if asyncio.get_event_loop().is_running():
            # We're in Jupyter - nest_asyncio should allow this
            asyncio.create_task(self._run_agent(agent_obj, name, port, description, api_token))
        else:
            # Not in a running loop, create and start one
            asyncio.run(self._run_agent(agent_obj, name, port, description, api_token))
    
    def stop_agent(self):
        """Stop the agent and clean up resources"""
        if self.running and self.agent_name:
            print(f"\nStopping agent {self.agent_name}...")
            self.running = False
            self.stop_event.set()
            try:
                cleanup_uagent(self.agent_name)
                print(f"Agent {self.agent_name} stopped.")
                clear_output(wait=True)
                display(HTML(f"""
                <div style="padding: 10px; border: 2px solid #FF5722; border-radius: 5px; margin: 10px 0;">
                    <h3 style="color: #FF5722;">✓ Agent Terminated</h3>
                    <p>The agent <b>{self.agent_name}</b> has been successfully stopped.</p>
                </div>
                """))
                return True
            except Exception as e:
                print(f"Error stopping agent: {str(e)}")
                return False
        return False

# Create a global runner
agent_runner = AgentRunner()

# Function to display agent information
def show_agent_info():
    if agent_runner.agent_info:
        agent_address = agent_runner.agent_info.get('address', 'unknown')
        agent_port = agent_runner.agent_info.get('port', 'unknown')
        
        print(f"Agent Name: {agent_runner.agent_name}")
        print(f"Agent Address: {agent_address}")
        print(f"Agent Port: {agent_port}")
        print(f"Agentverse URL: https://agentverse.ai/agents/details/{agent_address}")
        print(f"Local Inspector: https://agentverse.ai/inspect/?uri=http://127.0.0.1:{agent_port}&address={agent_address}")
        print("\nNOTE: The Local Inspector typically won't work in Jupyter environments because")
        print("local ports are not publicly accessible. Use the Agentverse URL instead.")
    else:
        print("No agent information available")

# Function to start the agent
def run_agent(agent_obj, name="calculator_agent_jupyter", port=8765, 
              description="Calculator agent for notebook", api_token=None):
    if api_token is None:
        api_token = API_TOKEN
        
    # Stop any existing agent
    if agent_runner.running:
        agent_runner.stop_agent()
    
    # Start the agent in the current event loop
    agent_runner.start_agent(agent_obj, name, port, description, api_token)
    return agent_runner

# Function to stop the agent
def stop_agent():
    return agent_runner.stop_agent()

## Creating a Simple Calculator Agent

Let's create a simple Langchain agent that can perform calculations.

In [16]:
# Define a simple calculator tool
def calculator_tool(expression: str) -> str:
    """Evaluates a basic math expression (e.g., '2 + 2 * 3')."""
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

# Create the Langchain tool and agent
tools = [
    Tool(
        name="Calculator",
        func=calculator_tool,
        description="Useful for evaluating math expressions"
    )
]

llm = ChatOpenAI(temperature=0, api_key=OPENAI_API_KEY)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

/var/folders/6w/50q5txxx1z57v4d3w9hkxby00000gp/T/ipykernel_81904/78101617.py:20: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


## Test the Langchain Agent

Let's test our Langchain agent to make sure it's working correctly.

In [19]:
# Test our Langchain agent locally
agent.invoke("What is 25 * 4 + 10?")



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I should use the Calculator tool to evaluate this math expression.
Action: Calculator
Action Input: "25 * 4 + 10"
Observation: 110
Thought:

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I now know the final answer
Final Answer: 110

> Finished chain.


{'input': 'What is 25 * 4 + 10?', 'output': '110'}

## Converting to a uAgent and Registering on Agentverse

Now let's convert our Langchain agent to a uAgent and register it on Agentverse.

In [22]:
# Start the agent as a uAgent
run_agent(
    agent_obj=agent,
    name="calculator_agent_notebook",
    port=8765,  # If this port is in use, try a different one like 8000
    description="A calculator agent for testing in a notebook",
    api_token=API_TOKEN
)

Starting agent 'calculator_agent_notebook' on port 8765...
Preferred port 8765 is in use, searching for alternative...
Port 8765 is already in use. Using alternative port 8000 instead.
INFO:     [calculator_agent_notebook]: Starting agent with address: agent1qfmjl36cyt8uzjyvqktjy08rnfc7gscmvwr8sxjf8chkg8s0l6adwu6yp23
INFO:     [calculator_agent_notebook]: Agent inspector available at https://agentverse.ai/inspect/?uri=http%3A//127.0.0.1%3A8000&address=agent1qfmjl36cyt8uzjyvqktjy08rnfc7gscmvwr8sxjf8chkg8s0l6adwu6yp23
INFO:     [calculator_agent_notebook]: Starting server on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     [calculator_agent_notebook]: Starting mailbox client for https://agentverse.ai


## Viewing Agent Information

You can check your agent information at any time.

In [31]:
# Show agent info anytime needed
show_agent_info()

Agent Name: calculator_agent_notebook
Agent Address: agent1qfmjl36cyt8uzjyvqktjy08rnfc7gscmvwr8sxjf8chkg8s0l6adwu6yp23
Agent Port: 8000
Agentverse URL: https://agentverse.ai/agents/details/agent1qfmjl36cyt8uzjyvqktjy08rnfc7gscmvwr8sxjf8chkg8s0l6adwu6yp23
Local Inspector: https://agentverse.ai/inspect/?uri=http://127.0.0.1:8000&address=agent1qfmjl36cyt8uzjyvqktjy08rnfc7gscmvwr8sxjf8chkg8s0l6adwu6yp23

NOTE: The Local Inspector typically won't work in Jupyter environments because
local ports are not publicly accessible. Use the Agentverse URL instead.


## Cleaning Up

When you're done, remember to stop your agent.

In [28]:
# Stop the agent when you're done
stop_agent()

True

## Important Notes About Local Access

- The "Could not find this Agent on your local host" error occurs because Jupyter environments don't typically expose local ports to the browser
- Your agent is still running and properly registered with Agentverse
- The best way to interact with your agent is through the Agentverse web interface
- For full local access, consider running your notebook on a local Jupyter installation rather than JupyterLab or cloud environments